# Using FABRIC GPUs

Your compute nodes can include GPUs. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use Nvidia GPU devices on FABRIC.


## Setup the Experiment

#### Import FABRIC API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Create a Node

The cell below creates a slice that contains a single node. The node includes a GPU component.

### Set the Slice Name and FABRIC Site


In [ ]:
slice_name="MySlice"
site = 'STAR'
node_name='Node1'

gpu_name='gpu1'

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=node_name, site=site)
    
    #Add a GPU 
    #node.add_component(model='GPU_TeslaT4', name=gpu_name)
    # OR
    node.add_component(model='GPU_RTX6000', name=gpu_name)

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Get the Slice

Retrieve the node information and save the management IP addresses.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Get the Node

Retrieve the node information and save the management IP address.


In [ ]:
try:
    node = slice.get_node(node_name) 
    print(f"{node}")
  
    nvme1 = node.get_component(gpu_name)
    print(f"{nvme1}")
    
except Exception as e:
    print(f"Exception: {e}")

### GPU PCI Device

Run the command <code>lspci</code> to see your GPU PCI device(s). This is the raw GPU PCI device that is not yet configured for use.  You can use the GPUs as you would any GPUs.

View node1's GPU

In [ ]:
command = "sudo dnf install -q -y pciutils && lspci | grep 'NVIDIA\|3D controller'"
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

## Install Nvidia Drivers

Now, let's run the following commands to install the latest CUDA driver and the CUDA libraries and compiler.

In [ ]:
commands = [
    'sudo dnf install -q -y epel-release',
    'sudo dnf config-manager --add-repo https://developer.download.nvidia.com/compute/cuda/repos/rhel8/x86_64/cuda-rhel8.repo',
    'sudo dnf install -q -y kernel-devel kernel-headers nvidia-driver nvidia-settings cuda-driver cuda'
]
try:
    print("Installing CUDA...")
    for command in commands:
        stdout, stderr = node.execute(command)
    print("Done installing CUDA. Now, reboot for the changes to take effect.")
except Exception as e:
    print(f"Fail: {e}")

And once CUDA is installed, reboot the machine.

In [ ]:
reboot = 'sudo reboot'
try:
    print(reboot)
    node.execute(reboot)
    
    slice.wait_ssh(timeout=360,interval=10,progress=True)

    print("Now testing SSH abilites to reconnect...",end="")
    slice.update()
    slice.test_ssh()
    print("Reconnected!")

except Exception as e:
    print(f"Fail: {e}")

## Testing the GPU and CUDA Installation

First, verify that the Nvidia drivers recognize the GPU by running `nvidia-smi`.

In [ ]:
try:
    stdout, stderr = node.execute("nvidia-smi")
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

Now, let's upload the following "Hello World" CUDA program file to the node.

`hello-world.cu`

*Source: https://computer-graphics.se/multicore/pdf/hello-world.cu*

*Author: Ingemar Ragnemalm*

>This file is from *"The real "Hello World!" for CUDA, OpenCL and GLSL!"* (https://computer-graphics.se/hello-world-for-cuda.html), written by Ingemar Ragnemalm, programmer and CUDA teacher. The only changes (if you download the original file from the website) are to additionally `#include <unistd.h>`, as `sleep()` is now a fuction defined in the `unistd.h` library.

In [ ]:
node.upload_file('./hello-world.cu', 'hello-world.cu')

We now compile the `.cu` file using `nvcc`, the CUDA compiler tool installed with CUDA. In this example, we create an executable called `hello_world`.

In [ ]:
try:
    stdout, stderr = node.execute("/usr/local/cuda-11.7/bin/nvcc -o hello_world hello-world.cu")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

Finally, run the executable:

In [ ]:
try:
    stdout, stderr = node.execute("./hello_world")
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

If you see `Hello World!`, the CUDA program ran successfully. `World!` was computed on the GPU from an array of offsets being summed with the string `Hello `, and the resut was printed to stdout.

### Congratulations! You have now successfully run a program on a FABRIC GPU!

## Cleanup Your Experiment

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")